# Low-rank approximation on $\mathcal{P}(d)$ - the space of $d$-dimensional SPD matrices

In this notebook we want to get some intuition in different approaches for computing low-rank approximations for manifold-valued signals

In [1]:
using Manifolds
using Manopt
using LinearAlgebra
using Random
using Plots
using LaTeXStrings
using BenchmarkTools

In [50]:
include("../../../src/decompositions/signals/naive_low_rank_approximation.jl")
include("../../../src/decompositions/signals/curvature_corrected_low_rank_approximation.jl")
include("../../../src/decompositions/signals/exact_low_rank_approximation.jl")

include("../../../src/functions/loss_functions/curvature_corrected_loss.jl")
include("../../../src/functions/loss_functions/exact_loss.jl")

exact_loss (generic function with 2 methods)

### Load data and construct manifold ###

In [3]:
# load data
M = SymmetricPositiveDefinite(3)
d = manifold_dimension(M)
n = 100  # 100


100

In [4]:
e = 1. * Matrix(I, 3, 3)
# compute basis
Θ = get_basis(M, e, DefaultOrthonormalBasis())
#  construct data
τ = 2.  # variance
σ = .05  # variance
Xₑ = Θ.data[4]
print(Xₑ)

Random.seed!(31)
predata = [exp(M, e, sqrt(τ) * randn(1)[1] * Xₑ) for i in 1:n]

data = [exp(M, predata[i], random_tangent(M, predata[i], Val(:Gaussian), σ)) for i in 1:n]; # ∈ P(3)^n


[0.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 0.0]

In [5]:
# Export slice image
num_export = 10
asymptote_export_SPD("results/artificial1D_orig.asy", data=data[1:min(num_export,n)], scale_axes=(2,2,2)); 

### Construct low rank approximation ###

In [5]:
q = mean(M, data)
log_q_data = log.(Ref(M), Ref(q), data);  # ∈ T_q P(3)^n

In [42]:
(eRr_q, eUr), costs = exact_low_rank_approximation(M, q, data, 2; stepsize=1/10000, max_iter=10)

(([[0.17525404284891413 -0.287957326702762 0.14872841784557383; -0.28795732670276203 -0.00010583998011175689 0.2667710267880373; 0.14872841784557386 0.2667710267880373 0.15378791440207504], [0.13209193428322943 0.09619096703363013 0.03361047693778811; 0.09619096703363016 -22.690965378102366 0.07797447309149758; 0.03361047693778812 0.07797447309149758 0.03476301769969326]], [0.04141185945717856 0.043589454988513694; -0.062377171919476435 0.06746220792835492; … ; 0.08428851980435229 -0.04426547676489212; 0.09977091731476738 0.09542655939149962]), [0.049940970415489165, 0.019723246782690336, 0.05663330241992408, 0.011140240157969297, 0.05555910194147775, 0.013479927280286783, 0.057924414731575044, 0.010207364781930284, 0.05457367466117646, 0.015238115241389493])

In [8]:
# costs

In [51]:
max_iter = 50

nR_q = []
nU = []
ccR_q = []
ccU = []
eR_q = []
eU = []
eCosts = []
for i in 1:d  
    println("#$(i) | computing naive low-rank approximation")
    nRr_q, nUr = naive_low_rank_approximation(M, q, data, i)
    push!(nR_q, nRr_q)
    push!(nU, nUr)
    println("#$(i) | computing curvature corrected low-rank approximation")
    ccRr_q, ccUr = curvature_corrected_low_rank_approximation(M, q, data, i); 
    push!(ccR_q, ccRr_q)
    push!(ccU, ccUr)
    println("#$(i) | computing exact low-rank approximation")
    (eRr_q, eUr), eCostsr = exact_low_rank_approximation(M, q, data, i; stepsize=1/100000, max_iter=max_iter); 
    push!(eR_q, eRr_q)
    push!(eU, eUr)
    push!(eCosts, eCostsr)
end

#1 | computing naive low-rank approximation
#1 | computing curvature corrected low-rank approximation
#1 | computing exact low-rank approximation
#2 | computing naive low-rank approximation
#2 | computing curvature corrected low-rank approximation
#2 | computing exact low-rank approximation
#3 | computing naive low-rank approximation
#3 | computing curvature corrected low-rank approximation
#3 | computing exact low-rank approximation
#4 | computing naive low-rank approximation
#4 | computing curvature corrected low-rank approximation
#4 | computing exact low-rank approximation
#5 | computing naive low-rank approximation
#5 | computing curvature corrected low-rank approximation
#5 | computing exact low-rank approximation
#6 | computing naive low-rank approximation
#6 | computing curvature corrected low-rank approximation
#6 | computing exact low-rank approximation


In [52]:
ref_distance = sum(distance.(Ref(M), Ref(q), data).^2)

naive_tangent_distances_r = zeros(d)
predicted_naive_distances_r= zeros(d)
true_naive_distances_r= zeros(d)

curvature_corrected_tangent_distances_r = zeros(d)
predicted_curvature_corrected_distances_r = zeros(d)
true_curvature_corrected_distances_r = zeros(d)

exact_tangent_distances_r = zeros(d)
exact_distances_r= zeros(d)

for rank in 1:d
    naive_log_q_data_r = Symmetric.([sum([nR_q[rank][i] * nU[rank][k,i] for i in 1:rank]) for k in 1:n])
    curvature_corrected_log_q_data_r = Symmetric.([sum([ccR_q[rank][i] * ccU[rank][k,i] for i in 1:rank]) for k in 1:n])
    exact_log_q_data_r = Symmetric.([sum([eR_q[rank][i] * eU[rank][k,i] for i in 1:rank]) for k in 1:n])
    
    # expoentiate back
    naive_data_r = exp.(Ref(M), Ref(q), naive_log_q_data_r)
    curvature_corrected_data_r = exp.(Ref(M), Ref(q), curvature_corrected_log_q_data_r)
    exact_data_r = exp.(Ref(M), Ref(q), exact_log_q_data_r)


    # compute relative tangent space error
    naive_tangent_distances_r[rank] = sum(norm.(Ref(M), Ref(q),  log_q_data - naive_log_q_data_r).^2) / ref_distance
    curvature_corrected_tangent_distances_r[rank] = sum(norm.(Ref(M), Ref(q),  log_q_data - curvature_corrected_log_q_data_r).^2) / ref_distance
    exact_tangent_distances_r[rank] = sum(norm.(Ref(M), Ref(q),  log_q_data - exact_log_q_data_r).^2) / ref_distance


    # compute relative manifold error
    predicted_naive_distances_r[rank] = curvature_corrected_loss(M, q, data, naive_log_q_data_r)
    true_naive_distances_r[rank] = exact_loss(M, q, data, naive_log_q_data_r)
    predicted_curvature_corrected_distances_r[rank] = curvature_corrected_loss(M, q, data, curvature_corrected_log_q_data_r)
    true_curvature_corrected_distances_r[rank] = exact_loss(M, q, data, curvature_corrected_log_q_data_r)
    exact_distances_r[rank] = exact_loss(M, q, data, exact_log_q_data_r)
    
end

In [53]:
# We want plots with (1) the lower bound error, (2) the actually uncorrected manifold error and (3) the corrected manifold error
plot(1:d, [naive_tangent_distances_r, true_naive_distances_r, true_curvature_corrected_distances_r, exact_distances_r], label = ["theoretical lower bound" "naive" "curvature corrected" "exact"], ylims=(0,1), xlims=(1,d),xaxis=("approximation rank"), yaxis=(L"$\varepsilon_{rel}$"))
savefig("results/artificial1D_errors_by_rank.png")
plot(1:d, [naive_tangent_distances_r .+ 1e-4, true_naive_distances_r .+ 1e-4, true_curvature_corrected_distances_r .+ 1e-4, exact_distances_r .+ 1e-4], label = ["theoretical lower bound" "naive" "curvature corrected" "exact"], ylims=(1e-4,1), xlims=(1,d), xaxis=("approximation rank"), yaxis=(L"$\varepsilon_{rel}$", :log), legend=:bottomleft)
savefig("results/artificial1D_logerrors_by_rank.png")
for i in 1:d-1
    if i == 1
        plot(1:length(eCosts[1]), eCosts[1], label = "rank 1", ylims=(1e-4,1), yaxis=(L"$\varepsilon_{rel}$", :log))
    else
        plot!(1:length(eCosts[i]), eCosts[i], label = "rank $(i)", ylims=(1e-4,1), yaxis=(L"$\varepsilon_{rel}$", :log))
    end
end
savefig("results/artificial1D_exact_iterate_loss.png")

"/Users/wdiepeveen/Documents/PhD/Projects/8 - Manifold-valued tensor decomposition/src/manifold-valued-tensors/experiments/1D/P3/results/artificial1D_exact_iterate_loss.png"

In [12]:
# It would be nice to also have a plot that tells us something about the error in predicting the manifold loss (using CCL) and the actual loss 
# (1) for the naive approach (2) for the curvature corrected approach
plot(1:d-1, (predicted_curvature_corrected_distances_r[1:end-1] .- true_curvature_corrected_distances_r[1:end-1] .+ 1e-16) ./ (curvature_corrected_tangent_distances_r[1:end-1] .* sqrt.(curvature_corrected_tangent_distances_r[1:end-1] .* ref_distance) .+ 1e-16), label=("curvature corrected"), xlims=(1,d-1),xaxis=("approximation rank"), yaxis=(L"$\delta_{rel}$"), color=3)
savefig("results/artificial1D_discrepancy_by_rank.png")

"/Users/wdiepeveen/Documents/PhD/Projects/8 - Manifold-valued tensor decomposition/src/manifold-valued-tensors/experiments/1D/P3/results/artificial1D_discrepancy_by_rank.png"

### Benchmark different methods ###

In [13]:
# t = @benchmark naive_low_rank_approximation(M, q, data, 2)
# mean(t).time
# std(t).time
# # TODO also do the benchmarking in here similarly as run above | also do this for 3 different step sizes for exact | if benchmark=true
# @benchmark exact_low_rank_approximation(M, q, data, 2; stepsize=1/10000, max_iter=50) 
# @benchmark curvature_corrected_low_rank_approximation(M, q, data, 2) 

In [14]:
@benchmark naive_low_rank_approximation(M, q, data, 2)

BenchmarkTools.Trial: 268 samples with 1 evaluation.
 Range (min … max):  15.700 ms … 51.748 ms  ┊ GC (min … max): 0.00% … 11.43%
 Time  (median):     17.460 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   18.689 ms ±  3.379 ms  ┊ GC (mean ± σ):  2.40% ±  6.26%

     ▁█▆█▅▁▃ ▂                                                 
  ▃▃▆█████████▅▆▅▆▃▃▄▃▃▁▃▃▄▃▄▃▁▃▁▃▁▃▄▃▁▅▄▃▃▄▃▃▄▃▃▃▃▃▃▁▃▃▁▁▁▃▃ ▃
  15.7 ms         Histogram: frequency by time          27 ms <

 Memory estimate: 4.95 MiB, allocs estimate: 54122.

In [15]:
@benchmark curvature_corrected_low_rank_approximation(M, q, data, 2) 

BenchmarkTools.Trial: 145 samples with 1 evaluation.
 Range (min … max):  31.764 ms … 41.607 ms  ┊ GC (min … max): 0.00% … 14.08%
 Time  (median):     33.763 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   34.644 ms ±  2.347 ms  ┊ GC (mean ± σ):  2.78% ±  5.62%

       ▁ ▁ ▄▃▇█▃▂    ▂                                         
  ▃▃▃▄▆█████████████▄█▄█▃▁▁▁▁▄▃▃▁▁▁▃▁▁▁▁▁▁▁▁▁▁▃▁▁▃▄▆▇▆▃▁▆▁▄▃▃ ▃
  31.8 ms         Histogram: frequency by time        40.7 ms <

 Memory estimate: 10.16 MiB, allocs estimate: 107620.

In [8]:
@benchmark exact_low_rank_approximation(M, q, data, 2; stepsize=1/100000, max_iter=1) 

BenchmarkTools.Trial: 34 samples with 1 evaluation.
 Range (min … max):  128.829 ms … 234.162 ms  ┊ GC (min … max): 0.00% … 1.84%
 Time  (median):     146.369 ms               ┊ GC (median):    2.79%
 Time  (mean ± σ):   148.167 ms ±  16.295 ms  ┊ GC (mean ± σ):  1.98% ± 1.31%

         ▅▂▅█▅  ▂                                                
  ▅▁▁▅▁████████▅█▁▁▁▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅ ▁
  129 ms           Histogram: frequency by time          234 ms <

 Memory estimate: 39.29 MiB, allocs estimate: 329653.

In [13]:
@benchmark exact_low_rank_approximation(M, q, data, 2; stepsize=1/100000, max_iter=50)  

BenchmarkTools.Trial: 4 samples with 1 evaluation.
 Range (min … max):  1.278 s …   1.404 s  ┊ GC (min … max): 3.20% … 2.89%
 Time  (median):     1.314 s              ┊ GC (median):    3.10%
 Time  (mean ± σ):   1.328 s ± 58.020 ms  ┊ GC (mean ± σ):  2.97% ± 0.29%

  █   █                       █                           █  
  █▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  1.28 s         Histogram: frequency by time         1.4 s <

 Memory estimate: 477.83 MiB, allocs estimate: 3797008.

In [24]:
nT = []
nΣ = []

for i in 1:d
    nbm = @benchmark naive_low_rank_approximation(M, q, data, $i)
    push!(nT, mean(nbm).time / 1e9)
    push!(nΣ, std(nbm).time / 1e9)
end

In [26]:
ccT = []
ccΣ = []

for i in 1:d
    ccbm = @benchmark curvature_corrected_low_rank_approximation(M, q, data, $i)
    push!(ccT, mean(ccbm).time / 1e9)
    push!(ccΣ, std(ccbm).time / 1e9)
end

In [61]:
eT1= []
eΣ1 = []

for i in 1:d
    ebm1 = @benchmark exact_low_rank_approximation(M, q, data, $i; stepsize=1/100000, max_iter=1)
    push!(eT1, mean(ebm1).time / 1e9)
    push!(eΣ1, std(ebm1).time / 1e9)
end

In [28]:
eT= []
eΣ = []

for i in 1:d
    ebm = @benchmark exact_low_rank_approximation(M, q, data, $i; stepsize=1/100000, max_iter=50)
    push!(eT, mean(ebm).time / 1e9)
    push!(eΣ, std(ebm).time / 1e9)
end

In [75]:
# write latex table 
for i in 1:d
        println("$(i) & " * raw"$" * "$(Float16(nT[i]))" * raw"\pm" * "$(Float16(nΣ[i]))" * raw"$" * " & " * raw"$" * "$(Float16(ccT[i])) " * raw"\pm" * " $(Float16(ccΣ[i]))" * raw"$" * " & " * raw"$" * "$(Float16(eT1[i])) " * raw"\pm" * " $(Float16(eΣ1[i]))" * raw"$" * " & " * raw"$" * "$(Float16(eT[i])) " * raw"\pm" * " $(Float16(eΣ[i]))" * raw"$ " * raw"\\ ")
end

for i in 1:d
        println("$(i) & " * raw"$" * "$(Float16(nT[i]))" * raw"$" * " & " * raw"$" * "$(Float16(ccT[i]))" * raw"$" * " & " * raw"$" * "$(Float16(eT1[i]))" * raw"$" * " & " * raw"$" * "$(Float16(eT[i]))" * raw"$ " * raw"\\ ")
end


1 & $0.01758\pm0.001916$ & $0.03433 \pm 0.0083$ & $0.1271 \pm 0.01959$ & $2.238 \pm 0.006622$ \\ 
2 & $0.01747\pm0.001852$ & $0.03802 \pm 0.007565$ & $0.1129 \pm 0.003098$ & $2.514 \pm 0.02917$ \\ 
3 & $0.01741\pm0.001665$ & $0.0457 \pm 0.004898$ & $0.1229 \pm 0.00485$ & $2.848 \pm 0.0486$ \\ 
4 & $0.01903\pm0.004562$ & $0.04764 \pm 0.003735$ & $0.1411 \pm 0.01698$ & $3.137 \pm 0.02524$ \\ 
5 & $0.01779\pm0.001828$ & $0.05722 \pm 0.005764$ & $0.1509 \pm 0.01746$ & $3.732 \pm 0.05893$ \\ 
6 & $0.01965\pm0.003826$ & $0.06122 \pm 0.005917$ & $0.0884 \pm 0.005333$ & $0.0868 \pm 0.00744$ \\ 
1 & $0.01758$ & $0.03433$ & $0.1271$ & $2.238$ \\ 
2 & $0.01747$ & $0.03802$ & $0.1129$ & $2.514$ \\ 
3 & $0.01741$ & $0.0457$ & $0.1229$ & $2.848$ \\ 
4 & $0.01903$ & $0.04764$ & $0.1411$ & $3.137$ \\ 
5 & $0.01779$ & $0.05722$ & $0.1509$ & $3.732$ \\ 
6 & $0.01965$ & $0.06122$ & $0.0884$ & $0.0868$ \\ 


In [84]:
# methods above each other and results per rank in colums
println("tHOSVD" * prod([" & " * raw"$" * "$(Float16(nT[i]))" * raw"$" for i in 1:d]) * raw"\\ ")
println("CC-tHOSVD" * prod([" & " * raw"$" * "$(Float16(ccT[i]))" * raw"$" for i in 1:d]) * raw"\\ ")
println("MC-tHOSVD (1 iteration)" * prod([" & " * raw"$" * "$(Float16(eT1[i]))" * raw"$" for i in 1:d]) * raw"\\ ")
println("MC-tHOSVD (50 iteration)" * prod([" & " * raw"$" * "$(Float16(eT[i]))" * raw"$" for i in 1:d]) * raw"\\ ")

tHOSVD & $0.01758$ & $0.01747$ & $0.01741$ & $0.01903$ & $0.01779$ & $0.01965$\\ 
CC-tHOSVD & $0.03433$ & $0.03802$ & $0.0457$ & $0.04764$ & $0.05722$ & $0.06122$\\ 
MC-tHOSVD (1 iteration) & $0.1271$ & $0.1129$ & $0.1229$ & $0.1411$ & $0.1509$ & $0.0884$\\ 
MC-tHOSVD (50 iteration) & $2.238$ & $2.514$ & $2.848$ & $3.137$ & $3.732$ & $0.0868$\\ 
